In [ ]:
#default_exp tta
from nbdev.showdoc import show_doc

# Test Time Augmentation

> Code adapted from https://github.com/qubvel/ttach.

In [ ]:
#hide
from fastcore.test import *

In [ ]:
#export
import torch
import itertools
from typing import List, Optional, Union
from fastcore.foundation import store_attr

## Functional

In [ ]:
#export
@torch.jit.script
def rot90(x:torch.Tensor, k:int=1):
    "rotate batch of images by 90 degrees k times"
    return torch.rot90(x, k, (2, 3))

@torch.jit.script
def hflip(x:torch.Tensor):
    "flip batch of images horizontally"
    return x.flip(3)

@torch.jit.script
def vflip(x:torch.Tensor):
    "flip batch of images vertically"
    return x.flip(2)

## Base Class

In [ ]:
#export
class BaseTransform(torch.nn.Module):
    identity_param = None
    def __init__(self, pname: str, params: Union[list, tuple]): 
        super(BaseTransform, self).__init__()
        store_attr()

In [ ]:
# Tests
tst_base = BaseTransform('x', (1,2))
torch.jit.script(tst_base)

RecursiveScriptModule(original_name=BaseTransform)

## Transform Classes

In [ ]:
#export
class HorizontalFlip(BaseTransform):
    "Flip images horizontally (left->right)"
    identity_param = False
    def __init__(self):
        super(HorizontalFlip, self).__init__("apply", [0, 1])

    def forward(self, x:torch.Tensor, apply:int=0, deaug:bool=False):
        if apply==1: x = hflip(x)
        return x

In [ ]:
# Tests
imgs = torch.randn(4, 2, 356, 356)
t = torch.jit.script(HorizontalFlip())
aug = t(imgs, 1, True)
deaug = t(aug, 1, False)
test_eq(imgs, deaug)

In [ ]:
#export
class VerticalFlip(BaseTransform):
    "Flip images vertically (up->down)"
    identity_param = False
    def __init__(self):
        super().__init__("apply", [0, 1])

    def forward(self, x:torch.Tensor, apply:int=0, deaug:bool=False):
        if apply==1: 
            x = vflip(x)
        return x

In [ ]:
# Tests
t = torch.jit.script(VerticalFlip())
aug = t(imgs, 1, True)
deaug = t(aug, 1, False)
test_eq(imgs, deaug)

In [ ]:
#export
class Rotate90(BaseTransform):
    "Rotate images 0/90/180/270 degrees (`angles`)"
    identity_param = 0
    def __init__(self, angles: List[int]):
        super().__init__("angle", angles)
        if self.identity_param not in angles:
            angles = [self.identity_param] + list(angles)
        
    @torch.jit.export
    def apply_aug_image(self, image:torch.Tensor, angle:int=0): #, **kwargs
        k = angle // 90 if angle >= 0 else (angle + 360) // 90
        #k = torch.div(angle, 90, rounding_mode='trunc') if angle >= 0 else torch.div((angle + 360), 90, rounding_mode='trunc')
        return rot90(image, k)
    
    def forward(self, x:torch.Tensor, angle:int=0, deaug:bool=False):
        return self.apply_aug_image(x, angle=-angle if deaug else angle)

In [ ]:
# Tests
t = torch.jit.script(Rotate90([180]))
aug = t(imgs, 90, False)
deaug = t(aug, 90, True)
test_eq(imgs, deaug)

## Pipeline Classes

In [ ]:
#export
class Chain(torch.nn.Module):
    def __init__(self, transforms: List[BaseTransform]):
        super().__init__()
        self.transforms = torch.nn.ModuleList(transforms)
        
    def forward(self, x, args:List[int], deaug:bool=False):
        for i, t in enumerate(self.transforms):
            x = t(x, args[i], deaug)
        return x

In [ ]:
# Tests
tfms=[HorizontalFlip(),VerticalFlip(), Rotate90(angles=[90,180,270])]
args = [1, 1, 90]

tst_chain = torch.jit.script(Chain(tfms))
tst_chain_deaug = torch.jit.script(Chain(tfms[::-1]))

aug = tst_chain(imgs, args, False)
deaug = tst_chain_deaug(aug, args[::-1], True)
test_eq(imgs, deaug)

In [ ]:
#export
class Transformer(torch.nn.Module):
    def __init__(self, transforms: List[BaseTransform], args:List[int]):
        super(Transformer, self).__init__()
        self.aug_pipeline = Chain(transforms)
        self.deaug_pipeline = Chain(transforms[::-1])
        self.args = args
        
    @torch.jit.export
    def augment(self, image:torch.Tensor):
        return self.aug_pipeline(image, self.args, deaug=False)

    @torch.jit.export
    def deaugment(self, mask:torch.Tensor):
        return self.deaug_pipeline(mask, self.args[::-1], deaug=True)

In [ ]:
# Tests
tst_tfm = Transformer(tfms, args)
torch.jit.script(tst_tfm)
aug = tst_tfm.augment(imgs)
deaug = tst_tfm.deaugment(aug)
test_eq(imgs, deaug)

In [ ]:
#export
class Compose(torch.nn.Module):
    def __init__(self, aug_transforms: List[BaseTransform]):
        super(Compose, self).__init__()
        self.transform_parameters = list(itertools.product(*[t.params for t in aug_transforms]))  
        self.items = torch.nn.ModuleList([Transformer(aug_transforms, args) for args in self.transform_parameters])

In [ ]:
# Tests
c = Compose(tfms)
c = torch.jit.script(c)
out = []
for t in c.items:
    aug = t.augment(imgs)
    deaug = t.deaugment(aug)
    out.append(deaug)
    test_eq(imgs, deaug)
out = torch.stack(out)
test_close(imgs, torch.mean(out, dim=0))

## Export -

In [ ]:
#hide
from nbdev.export import *
#notebook2script()
notebook2script(fname='07_tta.ipynb')

Converted 07_tta.ipynb.
